# Jonathan Halverson
# March 2021
# Users gaming the datascience nodes

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.style.use('halverson')

sacct -S 02/19 -P -a --partition=datascience -o jobid%20,user,account,state%12,start,elapsed,elapsedraw,timelimit%15,timelimitraw,cputimeraw,ncpus,nnodes,reqmem%10,partition%15,alloctres%50,qos,maxrss%15 > feb19_mar19_2021_datascience.csv

Read in the salloc data

In [ ]:
cols = ['Start']
kib = pd.read_csv('feb19_mar19_2021_datascience.csv', thousands=',', parse_dates=cols, delimiter="|")
kib.head(10)

In [ ]:
kib.shape

# Remove job steps to focus on JobID and Allocated Memory

In [ ]:
overall = kib[~kib.JobID.str.contains(".", regex=False)]

In [ ]:
overall.head(10)

In [ ]:
overall.shape

Check for duplicates:

In [ ]:
overall.JobID.unique().size

## NNodes

In [ ]:
assert overall[overall.NNodes != 1].shape[0] == 0
assert overall[overall.NNodes.isnull()].shape[0] == 0

## some cleaning of overall

In [ ]:
mem = overall[overall.AllocTRES.notnull()]
mem.shape

In [ ]:
assert mem[mem.AllocTRES.apply(lambda x: x.count(",") != 3)].shape[0] == 0 

In [ ]:
def convert_mem(x):
    parts = x.split(",")
    memreq = parts[2]
    assert "mem=" in memreq
    val = memreq[memreq.index("=") + 1:]
    assert "T" in val or "G" in val or "M" in val
    assert "." not in val  # check for 150.5G
    val = val.replace("M", "").replace("G", "000").replace("T", "000000")
    return int(float(val))

In [ ]:
mem['allocated'] = mem.AllocTRES.apply(convert_mem)
mem

# Get JobID and MaxRSS

In [ ]:
util = kib[["JobID", "MaxRSS"]]
util

In [ ]:
util = util[util.MaxRSS.notnull()]

In [ ]:
def remove_from_dot(x):
    if ("." in x):
        return x[:x.index(".")]
    else:
        return x

In [ ]:
util['jobid_collapsed'] = util.JobID.apply(remove_from_dot)

In [ ]:
util

In [ ]:
def remove_units(x):
    assert "K" in x or "M" in x
    if "." in x:
        x = x[:x.index(".")] + x[-1]
    x = int(float(x.replace("K", "").replace("M", "000")))
    return x / 1000  # units are megabytes with this division

In [ ]:
util['utilized'] = util.MaxRSS.apply(remove_units)

In [ ]:
util

In [ ]:
util = util[["jobid_collapsed", "utilized"]].groupby("jobid_collapsed").agg({"utilized":'max'})
util.columns = ["utilized"]
util

# Merge dataframes

In [ ]:
mem_util = pd.merge(mem, util, how='inner', left_on=['JobID'], right_on=['jobid_collapsed'])
mem_util["allocated"] = mem_util["allocated"] / 1000
mem_util["allocated_med"] = mem_util["allocated"]
mem_util["utilized"] = mem_util["utilized"] / 1000
mem_util["utilized_med"] = mem_util["utilized"]
mem_util["ratio"] = mem_util["utilized"] / mem_util["allocated"]
mem_util["ratio_med"] = mem_util["ratio"]
mem_util["forcount"] = 1
mem_util["CPUTimeRAW"] = mem_util["CPUTimeRAW"] / 3600
mem_util

In [ ]:
fields = ['User', 'Account', 'CPUTimeRAW', 'ratio', "ratio_med", "allocated", "utilized","allocated_med", "utilized_med", "forcount"]
actions = {'Account':'first', "forcount":"count", 'CPUTimeRAW':'sum', "utilized":"mean", "allocated":"mean", "utilized_med":"median", "allocated_med":"median", 'ratio':'mean', 'ratio_med':'median'}
cmb = mem_util[fields].groupby(by='User').agg(actions).sort_values(by="ratio").reset_index()
cmb.columns = ['NetID', 'Dept', 'Jobs', 'Total CPU-Hours', "Ave. Utilized (GB)", "Ave. Allocated (GB)", "Median Utilized (GB)", "Median Allocated (GB)", "ave(Utilized/Allocated)", "median(Utilized/Allocated)"]
cmb['Total CPU-Hours'] = cmb['Total CPU-Hours'].apply(round)
cmb['Ave. Utilized (GB)'] = cmb['Ave. Utilized (GB)'].apply(round)
cmb['Median Utilized (GB)'] = cmb['Median Utilized (GB)'].apply(round)
cmb['Ave. Allocated (GB)'] = cmb['Ave. Allocated (GB)'].apply(round)
cmb['Median Allocated (GB)'] = cmb['Median Allocated (GB)'].apply(round)
cmb['ave(Utilized/Allocated)'] = cmb['ave(Utilized/Allocated)'].round(2)
cmb['median(Utilized/Allocated)'] = cmb['median(Utilized/Allocated)'].round(2)

In [ ]:
cmb

In [ ]:
print(cmb[['NetID', 'Dept', 'Jobs', 'Total CPU-Hours', "Ave. Utilized (GB)", "Ave. Allocated (GB)", "ave(Utilized/Allocated)"]].to_string(index=False))